## Test SPIM data CEBRA model

- Use CEBRA time contrastive learning on neural data from one fish
    - design model
    - convert SPIM data to usable format 
    - load data
    - fit
    - plot embeddings
    - try to decode stimulus type (left/right spots)
        - create a discrete variable that labels the peri-stimulus frames for right and left spots
        - This should inform the decoder to separate embedding states (which should vary between left and right spots)<br/><br/>

In [2]:
import cebra
import numpy as np
import matplotlib.pyplot as plt
import os

In [7]:
# remember to create an array containing both variables
post_left_spot = np.arange(3)
post_right_spot = np.arange(3)
post_spot = np.column_stack([post_left_spot, post_right_spot])
post_spot.shape

(3, 2)